# Imports and data

In [1]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory as image_dataset 

import matplotlib.pyplot as plt
import numpy as np
import PIL
import pandas as pd
import pathlib
import os

2024-04-23 10:57:57.604178: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 10:57:57.664245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 10:57:57.664287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 10:57:57.666638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 10:57:57.674057: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 10:57:57.674662: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
data = pd.read_csv('data/set.csv').drop(columns=['DateTime'])
data

,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,datetimestamp
0,1360.0,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578,299706
1,1292.0,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255,299707
2,1402.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502,299708
3,1376.0,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867,299709
4,1272.0,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888,299710
...,...,...,...,...,...,...,...,...,...
8440,1314.0,1101.0,539.0,1374.0,1729.0,21.9,29.3,0.7568,309058
8441,1163.0,1027.0,604.0,1264.0,1269.0,24.3,23.7,0.7119,309059
8442,1142.0,1063.0,603.0,1241.0,1092.0,26.9,18.3,0.6406,309060
8443,1003.0,961.0,702.0,1041.0,770.0,28.3,13.5,0.5139,309061


# Results method

In [3]:
results = pd.DataFrame(columns=['Model', 'mse', 'mae', 'rmse', 'mape', 'r2'])
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


def appendError(model, testy, y_pred):
    model_name = str(type(model).__name__)
    mse = mean_squared_error(testy, y_pred)
    mae = mean_absolute_error(testy, y_pred)
    rmse = root_mean_squared_error(testy, y_pred)
    mape = mean_absolute_percentage_error(testy, y_pred)
    r2 = r2_score(testy, y_pred)
    return pd.DataFrame([{
        'Model': model_name,
        'mse': mse, 
        'mae': mae, 
        'mae': mae, 
        'rmse': rmse, 
        'mape': mape, 
        'r2': r2
    }])

In [4]:
def compare(ds: tf.data.Dataset, df: pd.DataFrame):
    print(df.iloc[0])
    for images in ds.take(1):
        # Plot the first image from the batch
        plt.figure()
        plt.imshow(images[0].numpy())  # Convert to numpy array and display as image
        plt.axis('off')  # Hide axis
        plt.show()

# Creating datasets

In [60]:
batch = 32
image_size = 224
data_dir = pathlib.Path('./imagesTML/textINimage/CO/').with_suffix('')
data_dir.glob
shuffle = False
seed = 123123
split = 0.2

In [89]:
y.values

array([1360., 1292., 1402., ..., 1142., 1003., 1071.])

In [98]:
X = data.drop(columns=['PT08.S1(CO)'])
y = data['PT08.S1(CO)']
y_ds = tf.data.Dataset.from_tensor_slices(y)
y_ds

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float64, name=None)>

In [68]:
whole_ds = image_dataset(
    data_dir,
    labels=None,
    image_size=(image_size, image_size),
    color_mode='grayscale',
    shuffle=False
)
whole_ds

Found 8445 files belonging to 1 classes.


<_PrefetchDataset element_spec=TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None)>

In [83]:
dataset = tf.data.Dataset.zip(whole_ds, y_ds)
dataset

<_ZipDataset element_spec=(TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>

In [85]:
print(len(list(dataset)))
print(len(list(whole_ds)))

264
264


In [81]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size).take(val_size)
test_ds = dataset.skip(train_size + val_size)

print(f"Train dataset size: {len(list(train_ds))}")
print(f"Validation dataset size: {len(list(val_ds))}")
print(f"Test dataset size: {len(list(test_ds))}")

Train dataset size: 211
Validation dataset size: 26
Test dataset size: 27


# Convoluting

In [94]:
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(image_size, image_size, 1)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])

In [95]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mse'])

In [96]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50
)

Epoch 1/50


ValueError: in user code:

    File "/home/miiky/progar/python/chamba/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/miiky/progar/python/chamba/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/miiky/progar/python/chamba/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/miiky/progar/python/chamba/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/miiky/progar/python/chamba/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/home/miiky/progar/p